In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd
from datetime import datetime
from utils.globals import FEATS_MAP

In [7]:
with open("data/processed_data/unique_feats_union.p", "rb") as file:  # Replace "file_name.p" with your file"s path
    uniq_feats = pickle.load(file)
print("biological_sex:", uniq_feats["biological_sex"])
print("infancy_vac:", uniq_feats["infancy_vac"])
print("ethnicity:", uniq_feats["ethnicity"])
print("race:", uniq_feats["race"])
uniq_feats.keys()

biological_sex: ['female', 'male']
infancy_vac: ['wp', 'ap']
ethnicity: ['unknown', 'hispanic or latino', 'not hispanic or latino']
race: ['more than one race', 'american indian/alaska native', 'native hawaiian or other pacific islander', 'unknown or not reported', 'black or african american', 'white', 'asian']


dict_keys(['isotype_antigen', 'cell_type_name', 'protein_id_olink', 'protein_id_legendplex', 'stimulation_tcell_activ', 'stimulation_tcell_polar', 'stimulation_protein', 'versioned_ensembl_gene_id', 'infancy_vac', 'biological_sex', 'ethnicity', 'race'])

# merged_metadata

In [17]:
years = [2023,2022,2021,2020]

def calculate_age(birthdate):
    birthdate = datetime.strptime(birthdate, "%Y-%m-%d")
    today = datetime.today()
    age = today.year - birthdate.year + (today.month - birthdate.month) / 12 + \
            (today.day - birthdate.day) / 365.25
    return age


for yr in years:
    dt_1 = "training_dataset"
    dt_2 = "LD"
    if yr == 2023:
        dt_1 = "challenge_dataset"
        dt_2 = "BD"
    specimen_dpth = f"data/3rd_challenge/raw_datasets/{dt_1}/{yr}{dt_2}_specimen.tsv"
    subject_dpth = f"data/3rd_challenge/raw_datasets/{dt_1}/{yr}{dt_2}_subject.tsv"
    specimen_meta = pd.read_csv(specimen_dpth, sep="\t")
    subject_meta = pd.read_csv(subject_dpth, sep="\t")
    merged_meta = pd.merge(specimen_meta, subject_meta, on="subject_id")

    merged_meta["day_diff_14"] = merged_meta["actual_day_relative_to_boost"].apply(lambda x: 14 - x)
    merged_meta["day_diff_1"] = merged_meta["actual_day_relative_to_boost"].apply(lambda x: 1 - x)
    merged_meta["day_diff_3"] = merged_meta["actual_day_relative_to_boost"].apply(lambda x: 3 - x)
    merged_meta["day_diff_30"] = merged_meta["actual_day_relative_to_boost"].apply(lambda x: 30 - x)
    
    merged_meta["year_of_birth"] = merged_meta["year_of_birth"].apply(calculate_age)
    
    merged_meta["biological_sex"] = merged_meta["biological_sex"].str.lower()
    merged_meta["biological_sex"] = merged_meta["biological_sex"].map(FEATS_MAP["biological_sex"])
    
    merged_meta["infancy_vac"] = merged_meta["infancy_vac"].str.lower()
    merged_meta["infancy_vac"] = merged_meta["infancy_vac"].map(FEATS_MAP["infancy_vac"])
    
    merged_meta["ethnicity"] = merged_meta["ethnicity"].str.lower()
    merged_meta["ethnicity"] = merged_meta["ethnicity"].map(FEATS_MAP["ethnicity"])
    
    merged_meta["race"] = merged_meta["race"].str.lower()
    merged_meta["race"] = merged_meta["race"].map(FEATS_MAP["race"])
    
    merged_meta.to_parquet(f"data/processed_data/{yr}_metadata.parquet", engine='pyarrow')
    
merged_meta.head()

,specimen_id,subject_id,actual_day_relative_to_boost,planned_day_relative_to_boost,specimen_type,visit,infancy_vac,biological_sex,ethnicity,race,year_of_birth,date_of_boost,dataset,day_diff_14,day_diff_1,day_diff_3,day_diff_30
0,1,1,-3,0,Blood,1,1,0,2,4,38.847023,2016-09-12,2020_dataset,17,4,6,33
1,2,1,1,1,Blood,2,1,0,2,4,38.847023,2016-09-12,2020_dataset,13,0,2,29
2,3,1,3,3,Blood,3,1,0,2,4,38.847023,2016-09-12,2020_dataset,11,-2,0,27
3,4,1,7,7,Blood,4,1,0,2,4,38.847023,2016-09-12,2020_dataset,7,-6,-4,23
4,5,1,11,14,Blood,5,1,0,2,4,38.847023,2016-09-12,2020_dataset,3,-10,-8,19


In [ ]:
raw_path = "data/3rd_challenge/raw_datasets"
